In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from time import sleep

In [3]:
from dechorate import constants
from dechorate.dataset import DechorateDataset, SyntheticDataset
from dechorate.utils.file_utils import save_to_pickle, load_from_pickle, save_to_matlab
from dechorate.utils.dsp_utils import normalize, envelope

In [4]:
data_dir = '../data/'
dataset_dir = data_dir + 'dECHORATE/'
path_to_processed = data_dir + 'processed/'
path_to_note_csv = dataset_dir + 'annotations/dECHORATE_database.csv'
path_to_after_calibration = path_to_processed + \
    'post2_calibration/calib_output_mics_srcs_pos.pkl'

note_dict = load_from_pickle(path_to_after_calibration)
dset = DechorateDataset(path_to_processed, path_to_note_csv)
sdset = SyntheticDataset()


datasets = constants['datasets']
c = constants['speed_of_sound']
Fs = constants['Fs']
recording_offset = constants['recording_offset']
L = constants['rir_length']

In [5]:
I = 30
J = 9
D = len(datasets)

## Extraction of Real and Sythetic RIRs

In [6]:
rirs_real = np.zeros([L, I, J, D])
rirs_synt = np.zeros([L, I, J, D])
mics = np.zeros([3, I])
srcs = np.zeros([3, J])


for d in tqdm(range(D), desc='Loop datasets'):
    for i in tqdm(range(I), desc='Lood mic', leave=False):
        for j in range(J):

            dataset_id = datasets[d]
            
            # get rir from the recondings
            dset.set_dataset(dataset_id)
            dset.set_entry(i, j)
            mic, src = dset.get_mic_and_src_pos()
            mics[:, i] = mic
            srcs[:, j] = src
            _, rrir = dset.get_rir()


            # get synthetic rir
            sdset = SyntheticDataset()
            sdset.set_room_size(constants['room_size'])
            sdset.set_dataset(dataset_id, absb=0.85, refl=0.15)
            sdset.set_c(c)
            sdset.set_k_order(17)
            sdset.set_mic(mics[0, i], mics[1, i], mics[2, i])
            sdset.set_src(srcs[0, j], srcs[1, j], srcs[2, j])
            _, srir = sdset.get_rir()
            Ls = len(srir)


            # measure after calibration
            rirs_real[:, i, j, d] = rrir[:L]
            rirs_synt[:Ls, i, j, d] = srir[:Ls]

print('done with the extraction')


done with the extraction


## Running Blaster
well lets see